In [1]:
import math
import os
import pathlib
import pickle
import sqlite3

from dotenv import load_dotenv

import numpy as np
import pandas as pd

# Neural network stuff
from darts import TimeSeries
from darts.utils.callbacks import TFMProgressBar
from darts.models import TCNModel, Prophet
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, r2_score, rmse, smape
from darts.utils.missing_values import fill_missing_values
from torch.nn import MSELoss

%load_ext dotenv
%dotenv

DATABASE_PATH = os.environ['DATABASE_PATH']
MODEL_STORE_PATH = os.environ['MODEL_STORE']

Python-dotenv could not parse statement starting at line 14


In [2]:
with sqlite3.connect(DATABASE_PATH) as conn:
    cur = conn.cursor()
    res = cur.execute("SELECT * FROM market_data where date >= '2022-09-01' order by ticker, date;")
    entries = res.fetchall()

full_dataset = pd.DataFrame(entries, columns=['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'stock_splits'])
full_dataset['date'] = pd.to_datetime(full_dataset['date'])
full_dataset['avg_price'] = full_dataset[['open', 'high', 'low', 'close']].mean(axis=1)
full_dataset['paid_dividends'] = full_dataset['dividends'].replace(0, np.nan)
full_dataset['period'] = full_dataset.date.dt.to_period('Q')
sample = full_dataset.query("ticker == 'FIBRAPL14.MX'").get(['date', 'avg_price'])

In [3]:
ts = TimeSeries.from_dataframe(df=sample, time_col='date', value_cols='avg_price', freq='B')
ts = fill_missing_values(ts, fill='auto')

In [4]:
model = Prophet()
model.fit(ts)

AttributeError: 'Prophet' object has no attribute 'stan_backend'